### 【 혼합 피쳐 + Pipeline  + GridSearchCV 】

[1] 모듈 로딩<hr>

In [ ]:
## =================================================
## 모듈 로딩
## =================================================
import pandas as pd
import numpy as np

## ML 데이터셋 관련
from sklearn.model_selection import train_test_split

## ML 전처리 관련
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

## ML 전처리 및 모델 연동 관련
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


## ML 학습 알고리즘 관련
from sklearn.linear_model import LogisticRegression


[2] 예제 데이터 생성<hr>

In [ ]:
## =================================================
##  1) 예제 데이터 만들기
## =================================================
##-> 전역 시드 고정 및 재현성 설정
np.random.seed(42)

##-> 샘플 수
n = 500

##=> DF 생성
df = pd.DataFrame({
    "age":      np.random.randint(20, 60, size=n),                    # 수치형
    "income":   np.random.normal(5000, 1500, size=n).round(0),        # 수치형
    "city":     np.random.choice(["Seoul", "Busan", "Incheon", "Daegu"], size=n, p=[0.5, 0.2, 0.2, 0.1]), # 범주형
    "job":      np.random.choice(["office", "engineer", "teacher", "student"], size=n),                   # 범주형
})

##-> 일부 결측치 추가 (전처리 확인용)
missing_idx = np.random.choice(n, size=40, replace=False)
df.loc[missing_idx[:20], "income"] = np.nan
df.loc[missing_idx[20:], "city"] = np.nan


##-> 타깃(target) 생성
logit = (
    0.06 * (df["age"] - 35)
    + 0.0007 * (df["income"].fillna(df["income"].median()) - 5000)
    + df["city"].fillna("Seoul").map({
        "Seoul": 0.6, "Busan": 0.2, "Incheon": 0.1, "Daegu": 0.0
    }).astype(float)
    + df["job"].map({
        "engineer": 0.5, "office": 0.2, "teacher": 0.1, "student": -0.2
    }).astype(float)
)

prob = 1 / (1 + np.exp(-logit))
df["target"] = (np.random.rand(n) < prob).astype(int)

print(df.head())
print("\nClass ratio(target=1):", df["target"].mean().round(3))

   age  income     city       job  target
0   58  4700.0      NaN   student       1
1   48  8037.0    Busan  engineer       1
2   34  3720.0  Incheon   teacher       0
3   27  4979.0    Busan    office       0
4   40  5303.0    Busan   teacher       1

Class ratio(target=1): 0.646


[3] 피쳐/타겟과 학습용/테스트용 데이터 분리<hr>

In [ ]:
## =========================================================
## [3-1] 피쳐와 타겟 X, y 분리
## =========================================================
X = df.drop(columns=["target"])
y = df["target"]

## =========================================================
## [3-2] 컬럼 지정
## =========================================================
num_cols = ["age", "income"]
cat_cols = ["city", "job"]


## =========================================================
## [3-3] 학습용/테스트용 분리
## =========================================================
X_train, X_test, y_train, y_test = train_test_split( X,
                                                     y,
                                                     test_size=0.2,
                                                     random_state=42,
                                                     stratify=y )


[4] 학습 준비<hr>

In [ ]:
## =========================================================
## [4-1] 전처리 파이프라인 (수치/범주 분리)
## =========================================================
## => 수치형
numeric_preprocess = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])

## => 범주형
categorical_preprocess = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore")),
])

## => 수치형 + 범주형
preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_preprocess,     num_cols),
        ("cat", categorical_preprocess, cat_cols),
    ]
)

[5] 학습 진행 :  <hr>

In [ ]:
## =========================================================
## [5-1]  모델 파이프라인 생성 및 학습/평가
## =========================================================
## => 모델 인스턴스 생성
model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("clf", LogisticRegression(max_iter=1000))
])

## => 학습 진행
model.fit(X_train, y_train)


,steps,"[('preprocess', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [ ]:
## =========================================================
## [5-2] 전처리 후 최종 피쳐 개수 확인
## =========================================================
Xt = model.named_steps["preprocess"].transform(X_train)
print("\n변환 후 X_train shape:", Xt.shape)


# 원핫 후 생성된 컬럼명까지 보고 싶으면(버전에 따라 지원):
try:
    feature_names = model.named_steps["preprocess"].get_feature_names_out()
    print(f"전처리 후 피쳐 이름들 :{len(feature_names)}개")
    print(f"샘플 피쳐 이름들\n{ feature_names[:15]}")
except Exception as e:
    print(f"get_feature_names_out() ERROR : {e}")


변환 후 X_train shape: (400, 10)
전처리 후 피쳐 이름들 :10개
샘플 피쳐 이름들
['num__age' 'num__income' 'cat__city_Busan' 'cat__city_Daegu'
 'cat__city_Incheon' 'cat__city_Seoul' 'cat__job_engineer'
 'cat__job_office' 'cat__job_student' 'cat__job_teacher']


In [ ]:
## =========================================================
## [5-3] 학습/테스트 데이터셋으로 평가
## =========================================================
print("\nTrain score:", model.score(X_train, y_train))
print("Test  score:",  model.score(X_test, y_test))


Train score: 0.7625
Test  score: 0.72


[6] 학습 진행 : GridSearchCV + Pipeline + ColumnTransformer + Model <hr>

In [ ]:
## =========================================================
## 모듈 로딩
## =========================================================
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression


## =========================================================
## 교차 검증 + 하이퍼파라미터 튜닝 + 데이터 누수 회피 진행
## =========================================================

##=> [6-1] 모델 인스턴스 생성
model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("clf", LogisticRegression(max_iter=1000))
])

##=> [6-2] GridSearchCV 하이퍼파라미터
param_grid = {
    #-> 모델 파라미터
    "clf__C": [0.01, 0.1, 1, 10],
    #-> OneHotEncoder 옵션도 튜닝 가능
    "preprocess__cat__ohe__min_frequency": [None, 5, 10],
}

## [예시] --- 튜닝할 파라미터 정의
# param_grid = {
#     # ===== 모델(분류기) 하이퍼파라미터 =====
#     "clf__C"     : [0.01, 0.1, 1, 10],
#     "clf__solver": ["lbfgs", "liblinear"],  # liblinear는 작은 데이터에 무난
#     # solver에 따라 penalty 제약이 있으니 간단히 l2 중심으로
#     "clf__penalty": ["l2"],

#     # ===== 전처리 파라미터(선택) =====
#     # 수치형 결측치 대체 방식 바꿔보기
#     "preprocess__num__imputer__strategy": ["median", "mean"],

#     # 범주형 OneHotEncoder의 희귀범주 처리(버전 따라 지원)
#     # 지원 안 하면 이 줄은 제거하세요.
#     "preprocess__cat__ohe__min_frequency": [None, 5, 10],
# }


##=> [6-3] GridSearchCV 인스턴스 생성
grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,
    scoring="accuracy",
    n_jobs=-1,
)

##=> [6-4] 교차 검증 + 하이퍼파라미터 튜닝 + 데이터 누수 회피
grid.fit(X_train, y_train)



##=> [6-5] 결과 추출
print(f"Best CV score: {grid.best_score_}" )
print(f"Best params:")
for k, v in grid.best_params_.items():
    print(f"  {k}: {v}")

print("\nTest score with best model:", grid.score(X_test, y_test))


Best CV score: 0.7549999999999999
Best params:
  clf__C: 0.1
  preprocess__cat__ohe__min_frequency: None

Test score with best model: 0.71
